In [1]:
import numpy as np
from keras.layers import Input, Conv2D, Dense, Flatten, Dropout, AveragePooling2D
from keras.models import Model
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from snntoolbox.io_utils.common import to_categorical

Using Theano backend.


Using cuDNN version 5005 on context None
Mapped name None to device cuda: GeForce GTX 980 Ti (0000:01:00.0)


In [2]:
weights = np.load('/home/rbodo/.snntoolbox/data/cifar10/xu/_80.06_epoch_83_wts_robust_normd.npy', encoding='bytes')

In [53]:
img_input = Input((3, 32, 32))
x = Conv2D(32, (5, 5), padding='same', activation='relu', weights=[np.transpose(weights[0], (2, 3, 1, 0))], use_bias=False)(img_input)
x = AveragePooling2D()(x)
x = Conv2D(32, (5, 5), padding='same', activation='relu', weights=[np.transpose(weights[1], (2, 3, 1, 0))], use_bias=False)(x)
x = AveragePooling2D()(x)
x = Conv2D(64, (5, 5), padding='same', activation='relu', weights=[np.transpose(weights[2], (2, 3, 1, 0))], use_bias=False)(x)    
x = AveragePooling2D()(x) 
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu', weights=[weights[3]], use_bias=False)(x)    
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax', weights=[weights[4]], use_bias=False)(x)    

model = Model(img_input, x)

model.compile('adam', 'categorical_crossentropy', ['accuracy'])

In [62]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_test = to_categorical(y_test, 10)

datagen = ImageDataGenerator(rescale=1./255, featurewise_center=True)
datagen.fit(x_test/255.)

testflow = datagen.flow(x_test, y_test, batch_size=len(x_test))
x_test, y_test = testflow.next()

filepath = '/home/rbodo/.snntoolbox/Datasets/cifar10/mean_subtracted/'
np.savez_compressed(filepath+'x_norm', x_test.astype('float32'))
np.savez_compressed(filepath+'x_test', x_test.astype('float32'))
np.savez_compressed(filepath+'y_test', y_test.astype('float32'))

In [61]:
score = model.evaluate_generator(testflow, 1)

GpuArrayException: b'out of memory'
Apply node that caused the error: GpuFromHost<None>(/input_51)
Toposort index: 2
Inputs types: [TensorType(float32, 4D)]
Inputs shapes: [(10000, 3, 32, 32)]
Inputs strides: [(12288, 4096, 128, 4)]
Inputs values: ['not shown']
Outputs clients: [[GpuContiguous(GpuFromHost<None>.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [31]:
score

[0.59299087971448894, 0.8125]

In [56]:
model.save('/home/rbodo/.snntoolbox/data/cifar10/xu/81.25.h5')